<a href="https://www.ired.info/"> <img src="file/ired_logo.png" alt="ired" style="width: 150px;"/> </a>

# Combined clssification model and motor control

### Initiate PyTorch

In [ ]:
import torch
import torchvision

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

### Select model for auto hand sanitizer

In [ ]:
model.load_state_dict(torch.load('ired_model.pth'))

### Load model and display camera

In [ ]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

##

import time
from adafruit_servokit import ServoKit
myKit = ServoKit (channels=16)
myKit.servo[15].angle = 180
myKit.servo[0].angle = 0

import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
press_slider = widgets.FloatSlider(description='press', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, press_slider]))

### Replace with your own threshold value (default: 0.5)

In [ ]:
threshold = 0.5

### Start the automatic hand hsanitizer 

The press action has a cooldown period of **5 seconds**.

In [ ]:
old_time=time.time()

def update(change):
    global press_slider, old_time, prob, threshold
    x = change['new'] 
    x = preprocess(x)
    y = model(x)

    y = F.softmax(y, dim=1)
    
    prob_press = float(y.flatten()[0])
    press_slider.value = prob_press
    
    if prob_press > threshold:
        if time.time()- old_time > 5:
            myKit.servo[15].angle = 45
            myKit.servo[0].angle = 135
            time.sleep(0.5)
            myKit.servo[15].angle = 180
            myKit.servo[0].angle = 0
            old_time=time.time()
    
    time.sleep(0.001)
        
update({'new': camera.value})

camera.observe(update, names='value') 

#### Congratulations, you have successfully made an automatic hand sanitizer !!

### Stop auto hand sanitizer and camera

In [ ]:
camera.stop()

---End---